In [1]:
import melt_functions as ice_melt
import numpy as np
import xarray as xr
import scipy.io as sio
from plot_icebergshape import plot_icebergshape
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from matplotlib import cm,colors
import pickle
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import pandas as pd
# import dataframe_image as dfi

In [2]:
melange_area_path = '/home/m484s199/iceberg_py/dev/geoms/helheim/melange_area/'
mel_list = sorted( [file for file in os.listdir(melange_area_path) if file.endswith('gpkg') ] )
date_list = [gpd.pd.to_datetime(file[8:18]) for file in mel_list]
gdf_list = [gpd.read_file(f'{melange_area_path}{file}') for file in mel_list]
    


In [3]:
mel_list

['helhiem_2016-04-24_area.gpkg',
 'helhiem_2018-05-04_area.gpkg',
 'helhiem_2020-09-03_area.gpkg',
 'helhiem_2023-07-27_area.gpkg',
 'helhiem_2024-08-22_area.gpkg']

In [4]:
date_list

[Timestamp('2016-04-24 00:00:00'),
 Timestamp('2018-05-04 00:00:00'),
 Timestamp('2020-09-03 00:00:00'),
 Timestamp('2023-07-27 00:00:00'),
 Timestamp('2024-08-22 00:00:00')]

In [5]:
gdf_list[0].area[0]

np.float64(132528531.77252682)

In [6]:
area_dict = {date:gdf.area[0] for date, gdf in zip(date_list, gdf_list)}
area_dict

{Timestamp('2016-04-24 00:00:00'): np.float64(132528531.77252682),
 Timestamp('2018-05-04 00:00:00'): np.float64(142934146.47607896),
 Timestamp('2020-09-03 00:00:00'): np.float64(141325864.68025035),
 Timestamp('2023-07-27 00:00:00'): np.float64(142934146.47607896),
 Timestamp('2024-08-22 00:00:00'): np.float64(134061538.63418496)}

In [7]:
dt = 150

psw = 1024 #kg m3
csw = 3974 #J kg-1 C-1
day2sec = 86400
# Volume_test = 158461041 * 450 # area from Helheim fjord shapefile ~ 28 km length
Volume_test = 142934146.47607803 * 450 # area from Helheim fjord shapefile ~ 28 km length
# 134061538.63418299
coeff_1_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'
coeff_4_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'

coeff_1_list = sorted([nc for nc in os.listdir(coeff_1_path) if nc.endswith('nc')])
coeff_4_list = sorted([nc for nc in os.listdir(coeff_4_path) if nc.endswith('nc')])

def Qaw_calc(area):

    dt = 150

    psw = 1024 #kg m3
    csw = 3974 #J kg-1 C-1
    day2sec = 86400
    depth = 450
    
    vol = area * depth
    
    Qaw = psw * csw * ( (vol * 6.3) / (dt * day2sec) )

    return Qaw

dQ_dt_HEL_CTD_avg_dict = {date:Qaw_calc(vol) for date, vol in area_dict.items()}

# dQ_dt_HEL_CTD_avg = psw * csw * ( (Volume_test * 5.4) / (dt * day2sec) )

os.chdir(coeff_1_path)

cols = ['coeff', 'urel', 'tf', 'dt', 'Qib', 'Qaww', 'melt_rate_avg_m3s', 'melt_rate_mday', 'percentage']

In [8]:
dQ_dt_HEL_CTD_avg_dict

{Timestamp('2016-04-24 00:00:00'): np.float64(117973718299.14082),
 Timestamp('2018-05-04 00:00:00'): np.float64(127236546773.49007),
 Timestamp('2020-09-03 00:00:00'): np.float64(125804892917.60652),
 Timestamp('2023-07-27 00:00:00'): np.float64(127236546773.49007),
 Timestamp('2024-08-22 00:00:00'): np.float64(119338364215.22423)}

In [9]:
dQ_dt_HEL_CTD_avg/1e9

NameError: name 'dQ_dt_HEL_CTD_avg' is not defined

In [10]:
coeff_1_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'
coeff_4_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'
coeff_1_list = sorted([nc for nc in os.listdir(coeff_1_path) if nc.endswith('nc')])
coeff_4_list = sorted([nc for nc in os.listdir(coeff_4_path) if nc.endswith('nc')])



cols = ['date', 'coeff', 'urel', 'tf', 'dt', 'Qib', 'Qaww', 'melt_rate_avg_m3s', 'percentage']
coef_1_dict = {}

series_list1 = []
for i,nc in enumerate(coeff_1_list):
    
    Qib = xr.open_dataset(f'{coeff_1_path}{nc}')
    Qib_val = Qib.Qib.data
    date = nc[:10]
    date = pd.to_datetime(date)
    TF = nc.split('_')[6]
    urel_val = nc.split('_')[-1].split('.')[1]
    
    # print(f'coeff 1: {percentage:.2f}')
    print(f'{date}')
    coef_1_dict['date'] = date
    coef_1_dict['coeff'] = 1
    coef_1_dict['urel'] = 0.15 #urels_dict[urel_val]
    coef_1_dict['tf'] = 6.3 #TF_dict[TF]
    coef_1_dict['dt'] = 150
    coef_1_dict['Qib'] = (Qib_val/1e11)
    # coef_1_dict['Qaww'] = dQ_dt_HEL_CTD_avg #(dQ_dt_dict[TF])
    coef_1_dict['Qaww'] = dQ_dt_HEL_CTD_avg_dict[date]
    
    coef_1_dict['melt_rate_avg_m3s'] = Qib.melt_rate_intergrated.data
    
    coef_1_dict['percentage'] = f'{(Qib_val/dQ_dt_HEL_CTD_avg_dict[date])*100:.2f}' #f'{(Qib_val/dQ_dt_dict[TF])*100:.2f}'
    
    series = pd.Series(coef_1_dict)
    series_list1.append(series)

df_1 = pd.DataFrame(series_list1, columns=cols)   

coef_4_dict = {}
series_list4 = []
for i,nc in enumerate(coeff_4_list):
    
    Qib = xr.open_dataset(f'{coeff_4_path}{nc}')
    Qib_val = Qib.Qib.data
    date = nc[:10]
    date = pd.to_datetime(date)
    TF_4 = nc.split('_')[6]
    urel_val = nc.split('_')[-1].split('.')[1]

    # print(f'coeff 4: {percentage:.2f}')
    print(f'{TF}')
    coef_4_dict['date'] = date
    coef_4_dict['coeff'] = 4
    coef_4_dict['urel'] = 0.15 #urels_dict[urel_val]
    coef_4_dict['tf'] = 6.3 #TF_dict[TF_4]
    coef_4_dict['dt'] = 150
    # coef_4_dict['Qib'] = f'{(Qib_val/1e11):.2f}'
    coef_4_dict['Qib'] = (Qib_val/1e11)
    # coef_4_dict['Qaww'] = (dQ_dt_HEL_CTD_avg/1e11) #(dQ_dt_dict[TF_4]/1e11)

    # coef_4_dict['Qaww'] = (dQ_dt_HEL_CTD_avg_dict[date]/1e11) #(dQ_dt_dict[TF_4]/1e11
    coef_4_dict['melt_rate_avg_m3s'] = Qib.melt_rate_intergrated.data

    
    coef_4_dict['percentage'] = f'{(Qib_val/dQ_dt_HEL_CTD_avg_dict[date])*100:.2f}'
    
    series4 = pd.Series(coef_4_dict)
    series_list4.append(series4)
    


df_1 = pd.DataFrame(series_list1, columns=cols)
df_4 = pd.DataFrame(series_list4, columns=cols)

df_concat = pd.concat([df_1, df_4],ignore_index=True)


2016-04-24 00:00:00
2018-05-04 00:00:00
2020-09-03 00:00:00
2023-07-27 00:00:00
2024-08-22 00:00:00
UREL
UREL
UREL
UREL
UREL


In [11]:
!pwd

/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min


In [12]:
nc.split('_')

['2024-08-22', 'helheim', 'coeff', '1', 'CTD', 'constant', 'UREL', '02.nc']

In [13]:
df_1

,date,coeff,urel,tf,dt,Qib,Qaww,melt_rate_avg_m3s,percentage
0,2016-04-24,1,0.15,6.3,150,0.019191,1.179737e+11,5.745698790409667,1.63
1,2018-05-04,1,0.15,6.3,150,0.019392,1.272365e+11,5.805985579973478,1.52
2,2020-09-03,1,0.15,6.3,150,0.021588,1.258049e+11,6.463498992504948,1.72
3,2023-07-27,1,0.15,6.3,150,0.046579,1.272365e+11,13.94585595047171,3.66
4,2024-08-22,1,0.15,6.3,150,0.035689,1.193384e+11,10.685202485398994,2.99


In [14]:
dt = 50

psw = 1024 #kg m3
csw = 3974 #J kg-1 C-1
day2sec = 86400
# Volume_test = 158461041 * 450 # area from Helheim fjord shapefile ~ 28 km length
# Volume_test = 142934146.47607803 * 450 # area from Helheim fjord shapefile ~ 28 km length
# 134061538.63418299
coeff_1_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'
coeff_4_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'

coeff_1_list = sorted([nc for nc in os.listdir(coeff_1_path) if nc.endswith('nc')])
coeff_4_list = sorted([nc for nc in os.listdir(coeff_4_path) if nc.endswith('nc')])

def Qaw_calc(area):

    dt = 50

    psw = 1024 #kg m3
    csw = 3974 #J kg-1 C-1
    day2sec = 86400
    depth = 450
    
    vol = area * depth
    
    Qaw = psw * csw * ( (vol * 4.0) / (dt * day2sec) )

    return Qaw

dQ_dt_HEL_CTD_avg_dict = {date:Qaw_calc(vol) for date, vol in area_dict.items()}

# dQ_dt_HEL_CTD_avg = psw * csw * ( (Volume_test * 5.4) / (dt * day2sec) )

os.chdir(coeff_1_path)

cols = ['coeff', 'urel', 'tf', 'dt', 'Qib', 'Qaww', 'melt_rate_avg_m3s', 'melt_rate_mday', 'percentage']

In [15]:
coeff_1_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'
coeff_4_path = '/home/m484s199/iceberg_py/dev/outfiles/iceberg_model_output/helheim/coeff_1_urel02_test_run_type_min/'
coeff_1_list = sorted([nc for nc in os.listdir(coeff_1_path) if nc.endswith('nc')])
coeff_4_list = sorted([nc for nc in os.listdir(coeff_4_path) if nc.endswith('nc')])



cols = ['date', 'coeff', 'urel', 'tf', 'dt', 'Qib', 'Qaww', 'melt_rate_avg_m3s', 'percentage']
coef_1_dict = {}

series_list1 = []
for i,nc in enumerate(coeff_1_list):
    
    Qib = xr.open_dataset(f'{coeff_1_path}{nc}')
    Qib_val = Qib.Qib.data
    date = nc[:10]
    date = pd.to_datetime(date)
    TF = nc.split('_')[6]
    urel_val = nc.split('_')[-1].split('.')[1]
    
    # print(f'coeff 1: {percentage:.2f}')
    print(f'{date}')
    coef_1_dict['date'] = date
    coef_1_dict['coeff'] = 1
    coef_1_dict['urel'] = 0.02 #urels_dict[urel_val]
    coef_1_dict['tf'] = 4.0 #TF_dict[TF]
    coef_1_dict['dt'] = 50
    coef_1_dict['Qib'] = (Qib_val/1e11)
    # coef_1_dict['Qaww'] = dQ_dt_HEL_CTD_avg #(dQ_dt_dict[TF])
    coef_1_dict['Qaww'] = dQ_dt_HEL_CTD_avg_dict[date]
    
    coef_1_dict['melt_rate_avg_m3s'] = Qib.melt_rate_intergrated.data
    
    coef_1_dict['percentage'] = f'{(Qib_val/dQ_dt_HEL_CTD_avg_dict[date])*100:.2f}' #f'{(Qib_val/dQ_dt_dict[TF])*100:.2f}'
    
    series = pd.Series(coef_1_dict)
    series_list1.append(series)

df_1 = pd.DataFrame(series_list1, columns=cols)   

coef_4_dict = {}
series_list4 = []
for i,nc in enumerate(coeff_4_list):
    
    Qib = xr.open_dataset(f'{coeff_4_path}{nc}')
    Qib_val = Qib.Qib.data
    date = nc[:10]
    date = pd.to_datetime(date)
    TF_4 = nc.split('_')[6]
    urel_val = nc.split('_')[-1].split('.')[1]

    # print(f'coeff 4: {percentage:.2f}')
    print(f'{TF}')
    coef_4_dict['date'] = date
    coef_4_dict['coeff'] = 4
    coef_4_dict['urel'] = 0.02 #urels_dict[urel_val]
    coef_4_dict['tf'] = 4.0 #TF_dict[TF_4]
    coef_4_dict['dt'] = 50
    # coef_4_dict['Qib'] = f'{(Qib_val/1e11):.2f}'
    coef_4_dict['Qib'] = (Qib_val/1e11)
    # coef_4_dict['Qaww'] = (dQ_dt_HEL_CTD_avg/1e11) #(dQ_dt_dict[TF_4]/1e11)

    # coef_4_dict['Qaww'] = (dQ_dt_HEL_CTD_avg_dict[date]/1e11) #(dQ_dt_dict[TF_4]/1e11
    coef_4_dict['melt_rate_avg_m3s'] = Qib.melt_rate_intergrated.data

    
    coef_4_dict['percentage'] = f'{(Qib_val/dQ_dt_HEL_CTD_avg_dict[date])*100:.2f}'
    
    series4 = pd.Series(coef_4_dict)
    series_list4.append(series4)
    


df_1 = pd.DataFrame(series_list1, columns=cols)
df_4 = pd.DataFrame(series_list4, columns=cols)

df_concat = pd.concat([df_1, df_4],ignore_index=True)


2016-04-24 00:00:00
2018-05-04 00:00:00
2020-09-03 00:00:00
2023-07-27 00:00:00
2024-08-22 00:00:00
UREL
UREL
UREL
UREL
UREL


In [16]:
df_1

,date,coeff,urel,tf,dt,Qib,Qaww,melt_rate_avg_m3s,percentage
0,2016-04-24,1,0.02,4.0,50,0.019191,2.247118e+11,5.745698790409667,0.85
1,2018-05-04,1,0.02,4.0,50,0.019392,2.423553e+11,5.805985579973478,0.80
2,2020-09-03,1,0.02,4.0,50,0.021588,2.396284e+11,6.463498992504948,0.90
3,2023-07-27,1,0.02,4.0,50,0.046579,2.423553e+11,13.94585595047171,1.92
4,2024-08-22,1,0.02,4.0,50,0.035689,2.273112e+11,10.685202485398994,1.57
